根据提示

> Hint: XOR the ciphertexts together, and consider what happens when a space is XORed with a character in [a-zA-Z].

首先需要研究空格和字母的异或运算结果，然后再对密文进行异或运算，研究规律。

In [3]:
import string 
space = ' '
letter = string.ascii_letters

print(ord(space))
print(letter)
print(''.join([chr(ord(i)^ord(space)) for i in letter]))

# 32
# abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
# ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

# 可以看出，空格和字母异或后，大小写会互换

32
abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


由于
$$
c_1 \oplus c_2 = m_1 \oplus k \oplus m_2 \oplus k = m_1 \oplus m_2
$$
对任意两个密文进行异或，结果为明文的异或结果，而明文中存在空格，据此可以通过异或结果还原出部分明文。


In [4]:
import sys

ciphertexts=[  
"315c4eeaa8b5f8aaf9174145bf43e1784b8fa00dc71d885a804e5ee9fa40b16349c146fb778cdf2d3aff021dfff5b403b510d0d0455468aeb98622b137dae857553ccd8883a7bc37520e06e515d22c954eba5025b8cc57ee59418ce7dc6bc41556bdb36bbca3e8774301fbcaa3b83b220809560987815f65286764703de0f3d524400a19b159610b11ef3e",  
"234c02ecbbfbafa3ed18510abd11fa724fcda2018a1a8342cf064bbde548b12b07df44ba7191d9606ef4081ffde5ad46a5069d9f7f543bedb9c861bf29c7e205132eda9382b0bc2c5c4b45f919cf3a9f1cb74151f6d551f4480c82b2cb24cc5b028aa76eb7b4ab24171ab3cdadb8356f",  
"32510ba9a7b2bba9b8005d43a304b5714cc0bb0c8a34884dd91304b8ad40b62b07df44ba6e9d8a2368e51d04e0e7b207b70b9b8261112bacb6c866a232dfe257527dc29398f5f3251a0d47e503c66e935de81230b59b7afb5f41afa8d661cb",  
"32510ba9aab2a8a4fd06414fb517b5605cc0aa0dc91a8908c2064ba8ad5ea06a029056f47a8ad3306ef5021eafe1ac01a81197847a5c68a1b78769a37bc8f4575432c198ccb4ef63590256e305cd3a9544ee4160ead45aef520489e7da7d835402bca670bda8eb775200b8dabbba246b130f040d8ec6447e2c767f3d30ed81ea2e4c1404e1315a1010e7229be6636aaa",  
"3f561ba9adb4b6ebec54424ba317b564418fac0dd35f8c08d31a1fe9e24fe56808c213f17c81d9607cee021dafe1e001b21ade877a5e68bea88d61b93ac5ee0d562e8e9582f5ef375f0a4ae20ed86e935de81230b59b73fb4302cd95d770c65b40aaa065f2a5e33a5a0bb5dcaba43722130f042f8ec85b7c2070",  
"32510bfbacfbb9befd54415da243e1695ecabd58c519cd4bd2061bbde24eb76a19d84aba34d8de287be84d07e7e9a30ee714979c7e1123a8bd9822a33ecaf512472e8e8f8db3f9635c1949e640c621854eba0d79eccf52ff111284b4cc61d11902aebc66f2b2e436434eacc0aba938220b084800c2ca4e693522643573b2c4ce35050b0cf774201f0fe52ac9f26d71b6cf61a711cc229f77ace7aa88a2f19983122b11be87a59c355d25f8e4",  
"32510bfbacfbb9befd54415da243e1695ecabd58c519cd4bd90f1fa6ea5ba47b01c909ba7696cf606ef40c04afe1ac0aa8148dd066592ded9f8774b529c7ea125d298e8883f5e9305f4b44f915cb2bd05af51373fd9b4af511039fa2d96f83414aaaf261bda2e97b170fb5cce2a53e675c154c0d9681596934777e2275b381ce2e40582afe67650b13e72287ff2270abcf73bb028932836fbdecfecee0a3b894473c1bbeb6b4913a536ce4f9b13f1efff71ea313c8661dd9a4ce",  
"315c4eeaa8b5f8bffd11155ea506b56041c6a00c8a08854dd21a4bbde54ce56801d943ba708b8a3574f40c00fff9e00fa1439fd0654327a3bfc860b92f89ee04132ecb9298f5fd2d5e4b45e40ecc3b9d59e9417df7c95bba410e9aa2ca24c5474da2f276baa3ac325918b2daada43d6712150441c2e04f6565517f317da9d3",  
"271946f9bbb2aeadec111841a81abc300ecaa01bd8069d5cc91005e9fe4aad6e04d513e96d99de2569bc5e50eeeca709b50a8a987f4264edb6896fb537d0a716132ddc938fb0f836480e06ed0fcd6e9759f40462f9cf57f4564186a2c1778f1543efa270bda5e933421cbe88a4a52222190f471e9bd15f652b653b7071aec59a2705081ffe72651d08f822c9ed6d76e48b63ab15d0208573a7eef027",  
"466d06ece998b7a2fb1d464fed2ced7641ddaa3cc31c9941cf110abbf409ed39598005b3399ccfafb61d0315fca0a314be138a9f32503bedac8067f03adbf3575c3b8edc9ba7f537530541ab0f9f3cd04ff50d66f1d559ba520e89a2cb2a83",  
"32510ba9babebbbefd001547a810e67149caee11d945cd7fc81a05e9f85aac650e9052ba6a8cd8257bf14d13e6f0a803b54fde9e77472dbff89d71b57bddef121336cb85ccb8f3315f4b52e301d16e9f52f904"  
]  
cArr = [
    [int(c[i:i+2], 16) for i in range(0, len(c), 2)] for c in ciphertexts
]

def strxor(a, b): 
    """xor two strings of different lengths"""  
    if len(a) > len(b):  
        return "".join([chr(ord(x) ^ ord(y)) for (x, y) in zip(a[:len(b)], b)])  
    else:  
        return "".join([chr(ord(x) ^ ord(y)) for (x, y) in zip(a, b[:len(a)])])

def arrxor(a, b):
    a = [int(a[i:i+2], 16) for i in range(0, len(a), 2)]
    b = [int(b[i:i+2], 16) for i in range(0, len(b), 2)]
    if len(a) > len(b):  
        return "".join([chr(x ^ y) for (x, y) in zip(a[:len(b)], b)])  
    else:  
        return "".join([chr(x ^ y) for (x, y) in zip(a, b[:len(a)])])


In [5]:
arrxor(ciphertexts[0], ciphertexts[1])

'\x12\x10L\x06\x13NW\t\x14\x0f\x10O\x02R\x1b\n\x04B\x02\x0cM\x07\x0b\x18OH\x15T\x1f\x08\x00HN\x1e\x02A\x06\x1d\x06MT\x0b\n\x02\x02\x10\x19E\x10\x16MO:\x00SC\x00NC\x0e\x1e\x1d\nRF\x12\x17\x1b\x01\x17\x00\x1b\x0eEC\x1c\x0c\x1d\x16\nR\r\x11tN\x19\x06\x1a\x11M\x0eU\x17O\x08NT7\x14\x05\x0b\x17CST\x1bH\x07\x0e\x00\x0eM'

In [6]:
possibleLetter = [[set([]) for _ in range(len(ciphertexts[i]))] for i in range(len(ciphertexts))]
# 第 i 个字符串和其它所有字符串在第j个位置异或的结果如果是字母，就添加进possibleLetter[i][j]对应的set里
for i in range(len(ciphertexts)):
    c0 = ciphertexts[i]
    for j in range(len(ciphertexts)):
        if i == j :
            continue
        c1 = ciphertexts[j]
        tep = arrxor(c0, c1)
        for k in range(min(len(c0), len(tep))):
            if tep[k] in letter:
                possibleLetter[i][k].add(tep[k].lower())
possibleLetter

[[{'w'},
  {'e'},
  {'e', 'h', 'l', 'u'},
  {'c'},
  {'a'},
  {'n'},
  {'a', 'c', 'n', 'o', 'p', 'v', 'w'},
  {'a'},
  {'a'},
  {'c'},
  {'t', 'y'},
  {'o'},
  {'r'},
  {'e', 'g', 'o', 'r', 's', 't', 'y'},
  {'t'},
  {'h'},
  {'e'},
  {'b', 'e', 'i', 'o', 'r'},
  {'n'},
  {'u'},
  {'m'},
  {'b', 'x'},
  {'e'},
  {'r'},
  {'b', 'h', 'i', 'o', 'r', 's', 'y'},
  {'a', 'h', 't'},
  {'a', 'e', 't', 'z'},
  {'a', 'o', 'q', 'r', 't'},
  {'w'},
  {'i'},
  {'t'},
  {'h', 'z'},
  {'a', 'g', 'h', 'k', 'm', 'n', 'p'},
  {'a', 'q'},
  {'c', 'o', 'u'},
  {'a', 'h'},
  {'c', 'n'},
  {'t'},
  {'u'},
  {'m'},
  {'a', 'f', 'n', 'r', 's', 't'},
  {'c'},
  {'o'},
  {'m'},
  {'p'},
  {'u'},
  {'t'},
  {'e'},
  {'r'},
  {'s'},
  {'g', 'k', 'm', 'o', 'z'},
  {'h', 'l', 'n', 'o', 'r', 't', 'w'},
  {'w'},
  {'e'},
  {'c', 'e', 'k', 'o', 's'},
  {'c'},
  {'a'},
  {'n'},
  {'b', 'c', 'd', 'e', 'k', 'm', 's', 'v'},
  {'a'},
  {'l'},
  {'s'},
  {'o'},
  {'a', 'e', 'r', 's', 'z'},
  {'f'},
  {'a'},
  {'c'},
  {'t'}

In [7]:
# 若某个集合内字母不同，则该位置可能为空格，将空格对应的密文都提取出来，便可进一步计算明文
treshhold = 6
spaceIndex = []
for i in range(len(possibleLetter)):
    for j in range(len(possibleLetter[i])):
        if len(possibleLetter[i][j]) > treshhold:
            spaceIndex.append((i, j))

In [8]:
# 将所有可能的空格提取出来
spaceCipher = [set([]) for _ in range(max([x[1] for x in spaceIndex])+1)]
for pp in spaceIndex:
    spaceCipher[pp[1]].add(cArr[pp[0]][pp[1]])
spaceCipher

[set(),
 set(),
 set(),
 set(),
 {233},
 set(),
 {248},
 {235},
 set(),
 set(),
 set(),
 {10},
 {237},
 {67},
 set(),
 {48},
 {14},
 set(),
 set(),
 set(),
 {138},
 {95},
 {205},
 {8},
 {128},
 set(),
 set(),
 set(),
 {173},
 {9},
 {229},
 set(),
 {73},
 {144},
 set(),
 set(),
 {52, 57},
 {216},
 set(),
 set(),
 set(),
 {188},
 set(),
 {80},
 {175},
 {160},
 {224},
 {70},
 {231},
 {67},
 set(),
 {208},
 {50},
 {17},
 set(),
 {237},
 {248},
 {200},
 {34},
 set(),
 set(),
 {137},
 {167},
 set(),
 {19},
 {125},
 set(),
 {220},
 set(),
 set(),
 {188},
 {99},
 {26},
 set(),
 {6},
 {171},
 {64},
 {159},
 set(),
 set(),
 {28},
 set(),
 set(),
 {48},
 {184},
 set(),
 set(),
 set(),
 {17},
 set(),
 {205},
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {78}]

In [9]:
spaceArr = []
for i in range(len(spaceCipher)):
    ss = spaceCipher[i]
    if len(ss) == 1:
        for hh in ss :
            spaceArr.append(hh)
    else:
        spaceArr.append(ord(' '))
# len(spaceCipher), len(spaceArr), spaceArr

In [10]:
# 看看已有的信息
len1cnt = 0
for i in spaceCipher:
    if len(i) == 1:
        len1cnt += 1
len(spaceCipher), len1cnt

(106, 52)

In [11]:
def xorDecArr(a, b):
    return [chr(a[i]^b[i]) for i in range(min(len(a), len(b)))]

''.join(xorDecArr(cArr[0], spaceArr))

'\x11|nÊA\x95\x00AÙ7aOR\x00ÁHE¯\x80-MBER\x00n~ÉWITC\x00QfÛWTÿ\r\x1aC"MPUTERSð\x00wEHCAN\x00\x91\x17SOwFAíT£\x87\x00TH.\x00NUM\x0cµR\x9ap\x15\x00ìwÎHaAÇüKä5v\x9d\x93K\x9c\x83ÈWcO'

In [12]:
# 若只有一个可能的字母，则大概率为明文
def xorOneHex(idx, a, b):
    i = idx * 2
    a = int(a[i:i+2], 16)
    b = int(b[i:i+2], 16)
    return chr(a ^ b)

def xorOneDec(a, b):
    return chr(a ^ b)

keySet = [set() for _ in range(max([len(cArr[i]) for i in range(len(possibleLetter))]))]

for i in range(len(possibleLetter)):
    for j in range(len(cArr[i])):
        possSet = possibleLetter[i][j]
        if len(possSet) == 1:
            for hh in possSet:
                keySet[j].add(xorOneHex(0, hex(ord(hh))[2:], hex(cArr[i][j])[2:]))

keySet

[{'F'},
 {'\x19', '9'},
 {'n'},
 {'\x89'},
 {'É'},
 {'Û', 'û'},
 {'Ø'},
 {'Ë'},
 {'\x98'},
 {'t'},
 {'5'},
 {'*'},
 {'Í'},
 {'C', 'c'},
 {'\x95'},
 {'\x10'},
 {'.'},
 {'¯'},
 {'Î'},
 {'X', 'x'},
 {'ª'},
 {'\x7f'},
 {'í'},
 {'\x08', '('},
 {'\xa0'},
 {'n'},
 {'k'},
 {'É'},
 {'\x8d'},
 {')'},
 {'Å'},
 {'\x0b'},
 {'i'},
 {'°'},
 set(),
 set(),
 {'\x19'},
 {'ø'},
 {'ª'},
 {'@'},
 {'\x1a'},
 {'\x9c'},
 {'m'},
 {'p'},
 {'\x8f'},
 {'\x80'},
 {'À'},
 {'f'},
 {'Ç'},
 {'c'},
 {'þ'},
 {'ð'},
 {'\x12', '2'},
 {'1'},
 {'H'},
 {'Í'},
 {'Ø', 'ø'},
 {'è'},
 {'\x02'},
 {'Ð'},
 {'['},
 {'©'},
 {'\x87'},
 {'w'},
 {'3'},
 {']'},
 {'®'},
 {'ü'},
 {'ì'},
 {'Õ'},
 {'\x9c'},
 {'C'},
 {':'},
 {'k'},
 {'&'},
 {'\x8b'},
 {'`'},
 {'¿'},
 {'N'},
 {'ð'},
 {'<'},
 {'\x9a'},
 {'a'},
 {'\x10'},
 {'\x98'},
 {'»'},
 set(),
 {'\x9a'},
 {'1'},
 {'a'},
 {'Í', 'í'},
 {'Ç', 'ç'},
 set(),
 {'\x04'},
 {'£'},
 {'5'},
 {'"'},
 {'Ï', 'ï'},
 {'Ò'},
 set(),
 {'Ò'},
 set(),
 {'\x8c'},
 {'W'},
 {'7'},
 {'n'},
 {'Û'},
 {'¨'},
 {'Â'},


In [13]:
key1cnt = 0
for i in keySet:
    if len(i) == 1:
        key1cnt += 1
print(key1cnt)

keyArr = []
for i in range(len(keySet)):
    ss = keySet[i]
    if len(ss) == 1:
        for hh in ss :
            keyArr.append(ord(hh))
    else:
        keyArr.append(ord(' '))

''.join(xorDecArr(keyArr, cArr[0]))

110


'w| ca\x95 aactorcthe n-mbez  5 with qfÛntum computers. ee c\x99n also factor the number 15 wwth ¬Çüog t\x9daKn\x83d to ba\x98\x1b t)r)e¡\x7fEmG\x7f - Óõ\x04er9 yA+1Ï\x1e'

In [14]:
# 还是有一些有多余的，，，看看密钥空间现在多大
ll = [len(i) for i in keySet]
ll
cnt = 1
for i in ll:
    if i != 0:
        cnt *= i
cnt
# 我超，8388608，肯定不行了，还有那么多为0的，mmp

8388608

In [15]:
# 不确定密钥的长度，看看密文的长度集合
[len(i) for i in cArr], max([len(i) for i in cArr])

([139, 112, 95, 144, 122, 172, 186, 127, 156, 95, 83], 186)

参考 https://github.com/Morrandir/Crypto001_Week1 感觉可以换个思路，专注于一个密文

In [16]:
[(idx, len(hh)) for idx, hh in enumerate(cArr)]

[(0, 139),
 (1, 112),
 (2, 95),
 (3, 144),
 (4, 122),
 (5, 172),
 (6, 186),
 (7, 127),
 (8, 156),
 (9, 95),
 (10, 83)]

In [42]:
target = cArr[-1]
visible = string.ascii_letters
from collections import defaultdict

hh = [defaultdict(int) for _ in range(max([len(hh) for hh in cArr]))]

for i in range(10):
    print('0'+str(i), end='|')
for i in range(10, 100):
    print(i, end='|')
print()

for index, i in enumerate(cArr):
    res = xorDecArr(target, i)
    tep = ''
    for cidx, c in enumerate(res):
        if c in visible:
            tep += c
            print(c, end = ' |')
        else:
            tep += '*'
            print('*', end = ' |') 
    for cidx, c in enumerate(tep):
        if c == '*':
            continue
        hh[cidx][c] += 1
    print()

print('---Plain---')
thesh = 3

for dic in hh:
    if not len(dic):
        print('*', end=' |')
    elif len(dic) >= thesh:
            print(' ', end = ' |')
    else:
        maxn = max([dic[j] for j in dic])
        for j in dic:
            if dic[j] == maxn:
                print(j.lower(), end = ' |')
                break


00|01|02|03|04|05|06|07|08|09|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|31|32|33|34|35|36|37|38|39|40|41|42|43|44|45|46|47|48|49|50|51|52|53|54|55|56|57|58|59|60|61|62|63|64|65|66|67|68|69|70|71|72|73|74|75|76|77|78|79|80|81|82|83|84|85|86|87|88|89|90|91|92|93|94|95|96|97|98|99|
* |* |E |C |* |* |C |* |* |* |T |* |* |S |* |* |* |E |N |* |* |X |E |* |H |T |* |* |* |* |* |* |G |Q |* |A |* |* |* |* |A |* |O |* |* |* |* |* |* |* |* |N |* |* |E |* |A |* |S |* |L |* |* |E |F |* |* |* |O |* |O |* |* |E |T |* |* |* |B |* |* |C |T |
* |* |* |E |* |E |* |* |* |* |D |M |* |* |* |* |* |* |L |* |S |* |N |* |* |* |N |T |* |* |* |N |* |O |* |* |* |* |* |E |* |* |E |* |* |* |* |E |* |I |C |* |* |* |* |R |A |U |* |* |R |* |* |* |* |* |* |* |N |* |O |* |* |* |* |* |* |* |T |* |N |N |E |
* |* |* |* |* |* |* |* |E |* |H |* |* |* |S |* |* |* |U |* |S |q |E |* |* |* |* |Q |U |* |* |N |* |O |* |* |* |* |R |* |* |* |P |* |* |* |* |* |* |D |E |* |* |V |* |* |N |U |* |* |I |* |* |E |A |K |*

In [18]:
hh

[defaultdict(int, {'t': 1}),
 defaultdict(int, {'H': 1}),
 defaultdict(int, {'E': 2, 'M': 1}),
 defaultdict(int, {'R': 4}),
 defaultdict(int, {'E': 1}),
 defaultdict(int, {'N': 2, 'I': 3, 'O': 1, 'c': 1, 'E': 1}),
 defaultdict(int, {'A': 2}),
 defaultdict(int, {'U': 1}),
 defaultdict(int, {'E': 1}),
 defaultdict(int, {'C': 1, 'L': 1, 'T': 2, 'R': 1, 'E': 2, 'I': 1}),
 defaultdict(int, {'T': 2, 'Y': 1}),
 defaultdict(int, {'W': 1}),
 defaultdict(int, {'O': 1}),
 defaultdict(int, {'R': 1, 'G': 1, 'T': 2, 'E': 1, 'Y': 1, 'o': 1, 'S': 1}),
 defaultdict(int, {'T': 4}),
 defaultdict(int, {'Y': 1}),
 defaultdict(int, {'P': 1}),
 defaultdict(int, {'E': 2}),
 defaultdict(int, {'S': 1}),
 defaultdict(int, {'U': 3, 'Y': 1, 'T': 2, 'C': 1, 'd': 1, 'I': 1}),
 defaultdict(int, {'O': 3}),
 defaultdict(int, {'F': 1}),
 defaultdict(int, {'E': 2, 'N': 1, 'D': 1, 'A': 1, 'H': 1, 'P': 1, 'T': 1}),
 defaultdict(int, {'C': 2}),
 defaultdict(int, {'Y': 1}),
 defaultdict(int, {'A': 1}),
 defaultdict(int, {'A'

In [20]:
# 创建一个 defaultdict，当键不存在时，返回默认值 0
d = defaultdict(int)

# 当我们尝试访问一个不存在的键时，它会返回默认值 0
print(d["some_key"])  # 输出：0

# 现在 "some_key" 已经在字典中了，它的值是 0
print(d)  # 输出：defaultdict(<class 'int'>, {'some_key': 0})


0
defaultdict(<class 'int'>, {'some_key': 0})



非常感谢您提供了这么详细的解题思路,我尝试用自然语言进行润色:

使用Python中的int、chr、join和base64等函数,可以轻松实现base64编码和解码。
与第一题类似,同样利用Python内置函数,实现十六进制和base64之间的转换。
通过暴力枚举所有可能的密钥,并设计一个评分函数作为判断标准,对解密结果进行评分,找到评分最高的密钥。
基于第三题的代码,进行暴力枚举攻击。需要注意,题目没有说明异或后的字符就是字母,这是一个陷阱。
调用之前实现的固定异或函数,但需要注意字符串与二进制、十六进制编码之间的转换需要对齐长度,不足补零。
首先实现汉明距离计算函数作为评分标准,读取base64编码的密文,按照给定方法计算不同密钥长度的评分,确定最有可能的密钥长度。然后对密文分组进行单字符异或破解,逐位得到密钥,并最终解密。
根据提示不建议使用OpenSSL,我直接用Python代码实现ECB模式的解密。
检测ECB模式是否使用的方法是,统计密文分组的重复出现频次,ECB模式下同一密文块重复加密会出现相同的密文,从而检测到ECB模式。

In [47]:
possbilePlaintext = 'The secret message is: When using a stream cipher, never use the key more than once'
possbilePlaintextArr = [ord(x) for x in possbilePlaintext]
key = [ord(x) for x in xorDecArr(possbilePlaintextArr, cArr[-1])]
for i in cArr:
    print(''.join(xorDecArr(key, i)))

We can factor the number 15 with quantum computers. We can also factor the number 1
Euler would probably enjoy that now his theorem becomes a corner stone of crypto - 
The nice thing about Keeyloq is now we cryptographers can drive a lot of fancy cars
The ciphertext produced by a weak encryption algorithm looks as good as ciphertext 
You don't want to buy a set of car keys from a guy who specializes in stealing cars
There are two types of cryptography - that which will keep secrets safe from your l
There are two types of cyptography: one that allows the Government to use brute for
We can see the point where the chip is unhappy if a wrong bit is sent and consumes 
A (private-key)  encryption scheme states 3 algorithms, namely a procedure for gene
 The Concise OxfordDictionary (2006) deï¬nes crypto as the art of  writing o r sol
The secret message is: When using a stream cipher, never use the key more than once
